# Neptune API tour


## Introduction

Vamos a ver cómo:

* Instalar el neptune-client
* Conectar Neptune a tu script y crear la primera corrida
* Registrar métricas simples y explorar en la interfaz de usuario UI
* Registrar curvas de aprendizaje, imágenes y modelos binarios desde el entrenamiento de Keras y verlo en la Neptune UI
* Solicitar los datos registrados a Neptune directamente desde la libreta y analizarlos

Al final, ejecutarás tu primer programa y lo verás en Neptune!


## Challenge:

Seleccionar una o dos libretas de las clases anteriores (preferentemente con deep learning)

Conectarse a neptune y ejecutar varios experimentos para determinar qué familia de parámetros nos da un buen performance para el modelo entrenado.


NOTA: recuerda correr al inicio de los experimentos el comando

```
run = neptune.init(
    project="username/projectname",
    api_token="token",
)  # your credentials
```

y al final desconecta el servicio con `run.stop()`

## Setup

Instalar el cliente Neptune

In [ ]:
! pip install neptune-client

     |████████████████████████████████| 276 kB 6.7 MB/s 
     |████████████████████████████████| 829 kB 42.4 MB/s 
     |████████████████████████████████| 52 kB 1.5 MB/s 
     |████████████████████████████████| 180 kB 65.3 MB/s 
     |████████████████████████████████| 131 kB 56.1 MB/s 
     |████████████████████████████████| 79 kB 8.8 MB/s 
     |████████████████████████████████| 8.2 MB 26.8 MB/s 
     |████████████████████████████████| 138 kB 52.3 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 127 kB 56.8 MB/s 
     |████████████████████████████████| 130 kB 59.9 MB/s 
     |████████████████████████████████| 67 kB 5.1 MB/s 
  Created wheel for neptune-client: filename=neptune_client-0.13.3-py2.py3-none-any.whl size=481965 sha256=3491190a7b3f11e891e9fd72062c507922bc73bd030f28f97db5e70e18e63029
  Stored in directory: /root/.cache/pip/wheels/0f/57/12/8fae2c4d320108451ca55c0c1288b69721f496062ed438d63d
  Created wheel for future: filena

## Initialize Neptune

Connect your script to Neptune application and create new run.

In [ ]:
import neptune.new as neptune

In [ ]:
run = neptune.init(
    project="username/projectname",
    api_token="token",
)  # your credentials

params = {"learning_rate": 0.001, "optimizer": "Adam"}
run["parameters"] = params

for epoch in range(10):
    run["train/loss"].log(0.9 ** epoch)

run["eval/f1_score"] = 0.66

run.stop()

Info (NVML): Driver Not Loaded. GPU usage metrics may not be reported. For more information, see https://docs-legacy.neptune.ai/logging-and-managing-experiment-results/logging-experiment-data.html#hardware-consumption 


https://app.neptune.ai/juanvazqmont/test01/e/TES-1
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.
Shutting down background jobs, please wait a moment...
Done!


Waiting for the remaining 23 operations to synchronize with Neptune. Do not kill this process.


All 23 operations synced, thanks for waiting!


In [ ]:
import neptune.new as neptune

run = neptune.init(project="common/colab-test-run", api_token="ANONYMOUS")

You tell Neptune: 

* **who you are**: your Neptune API token `api_token` 
* **where you want to send your data**: your Neptune `project`.

At this point you have new run in Neptune. For now on you will use `run` to log metadata to it.

---

**Note**


Instead of logging data to the public project 'common/colab-test-run' as an anonymous user 'neptuner' you can log it to your own project.

To do that:

1. Get your [Neptune API token](https://docs.neptune.ai/getting-started/installation#authentication-neptune-api-token)
2. Pass the token to ``api_token`` argument of ``neptune.init()`` method: ``api_token=YOUR_API_TOKEN``
3. Pass your project to the ``project`` argument of the ``neptune.init()``.

For example:

```python
neptune.init(project='my_workspace/my_project', 
             api_token='MY_API_TOKEN')
```

## Basic Example

Lets start with something super simple.

You will:
* log parameters
* log a training and validation scores
* log loss
* add name to the run
* append two tags

In [ ]:
params = {"learning_rate": 0.1}

# log params
run["parameters"] = params

# log name and append tags
run["sys/name"] = "basic-colab-example"
run["sys/tags"].add(["colab", "intro"])

# log loss during training
for epoch in range(100):
    run["train/loss"].log(0.99 ** epoch)

# log train and validation scores
run["train/accuracy"] = 0.95
run["valid/accuracy"] = 0.93

Runs can be viewed as dictionary-like structures - **namespaces** - that you can define in your code. You can apply hierarchical structure to your metadata that will be reflected in the UI as well. Thanks to this you can easily organize your metadata in a way you feel is most convenient.

There is one special namespace: **system namespace**, denoted `sys`. You can use it to add name and tags to the run.

Try to change the values and re-execute to see your runs appear in the dashboard.

All `basic-colab-example` runs are grouped in [this dashboard view](https://app.neptune.ai/o/common/org/colab-test-run/experiments?viewId=2eb46477-0064-4617-a182-17f8a2fcc06a).

![colab-runs](https://neptune.ai/wp-content/uploads/docs-api-tour.png)

There are many other things that you can log to neptune:

* Images and charts
* Artifacts like model weights or results
* Text values
* Hardware consumption
* Code snapshots
* and more

You can go and see all that in the [documentation](https://docs.neptune.ai/user-guides/logging-and-managing-runs-results/logging-runs-data) but you can check out the next example to see some of those.


#### Stop logging

<font color=red>**Warning:**</font><br>
Once you are done logging, you should stop tracking the run using the `stop()` method.
This is needed only while logging from a notebook environment. While logging through a script, Neptune automatically stops tracking once the script has completed execution.

In [ ]:
run.stop()

## Keras classification example [Advanced]

Install and import your machine learning libraries

In [ ]:
! pip install scikit-plot==0.3.7 tensorflow==2.4.1

Get the data

In [ ]:
import tensorflow as tf

mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

Prepare the model

In [ ]:
model = tf.keras.models.Sequential(
    [
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(512, activation="relu"),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation="softmax"),
    ]
)

model.compile(
    optimizer="sgd", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

To log metrics after every batch and epoch let's create `NeptuneLogger` callback:

In [ ]:
from tensorflow.keras.callbacks import Callback


class NeptuneLogger(Callback):
    def on_batch_end(self, batch, logs={}):
        for log_name, log_value in logs.items():
            run["batch/{}".format(log_name)].log(log_value)

    def on_epoch_end(self, epoch, logs={}):
        for log_name, log_value in logs.items():
            run["epoch/{}".format(log_name)].log(log_value)

Now we simply need to create a run.
I will tag it with the name `advanced` and log hyperparameters `epoch_nr` and `batch_size`: 

In [ ]:
EPOCH_NR = 5
BATCH_SIZE = 32

run = neptune.init(project="common/colab-test-run", api_token="ANONYMOUS")

# log params
run["parameters/epoch_nr"] = EPOCH_NR
run["parameters/batch_size"] = BATCH_SIZE

# log name and append tag
run["sys/name"] = "keras-metrics"
run["sys/tags"].add("advanced")

Now we pass our `NeptuneLogger` as keras callback and thats it.

In [ ]:
history = model.fit(
    x=x_train,
    y=y_train,
    epochs=EPOCH_NR,
    batch_size=BATCH_SIZE,
    validation_data=(x_test, y_test),
    callbacks=[NeptuneLogger()],
)

You can click on the run link above and monitor your learning curves as it is training!

![advanced](https://neptune.ai/wp-content/uploads/docs-api-tour-2.png)

Great thing is, you can log more things if you need to during or after the training is finished.

For example, let's calculate some additional metrics on test data and log them.

In [ ]:
import numpy as np

y_test_pred = np.asarray(model.predict(x_test))
y_test_pred_class = np.argmax(y_test_pred, axis=1)

In [ ]:
from sklearn.metrics import f1_score

run["test/f1"] = f1_score(y_test, y_test_pred_class, average="micro")

We can log diagnostic charts like confusion matrix or ROC AUC curve.

In [ ]:
import matplotlib.pyplot as plt
from scikitplot.metrics import plot_confusion_matrix, plot_roc

fig, ax = plt.subplots(figsize=(16, 12))
plot_confusion_matrix(y_test, y_test_pred_class, ax=ax)
run["diagnostic_charts"].log(neptune.types.File.as_image(fig))

fig, ax = plt.subplots(figsize=(16, 12))
plot_roc(y_test, y_test_pred, ax=ax)
run["diagnostic_charts"].log(neptune.types.File.as_image(fig))

![alt text](https://neptune.ai/wp-content/uploads/docs-api-tour-charts.png)

We can also log model weights to Neptune.

In [ ]:
model.save("my_model.h5")
run["model"].upload("my_model.h5")

![alt text](https://neptune.ai/wp-content/uploads/docs-api-tour-model.png)

With that you can share models with your teammates easily.

You can play around and execute this run with different parameters and see results and compare them.

Like I've done here:

![alt text](https://neptune.ai/wp-content/uploads/docs-api-tour-compare.png)

A cool thing is, once things are logged to Neptune you can access them from wherever you want.
Let me show you. 



## Access data you logged programatically 

## Getting the project's leaderboard

You can use the Project object to get aggregate data of all your runs. Here, fetch only runs with tag 'advanced'.

In [ ]:
my_project = neptune.get_project(name="common/colab-test-run", api_token="ANONYMOUS")
run_df = my_project.fetch_runs_table(tag=["advanced"]).to_pandas()
run_df.head()

#### Stop logging

<font color=red>**Warning:**</font><br>
Once you are done logging, you should stop tracking the run using the `stop()` method.
This is needed only while logging from a notebook environment. While logging through a script, Neptune automatically stops tracking once the script has completed execution.

In [ ]:
run.stop()

## Getting the run's metadata

Neptune lets you fetch whatever you logged to it directly to your notebooks and scripts.

In [ ]:
run = neptune.init(project="common/colab-test-run", api_token="ANONYMOUS", run="COL-7")

Now that your run is *fetched* you can download the metadata.

In [ ]:
batch_size = run["parameters/batch_size"].fetch()
last_batch_acc = run["batch/accuracy"].fetch_last()
print("batch_size: {}".format(batch_size))
print("last_batch_acc: {}".format(last_batch_acc))

You can even download artifacts from that run if you want to:



In [ ]:
run["model"].download()

In [ ]:
! ls ./

## Learn more about Neptune

Read about other Neptune features, create your free account and start logging!

[Go to Neptune](https://neptune.ai/register)